# Introduction to `esys.escript`

## Outline

This unit gives an introduction into solving partial differential equations (PDEs) in python. This 
section assumed that you have a basic understanding how to work with python.


We are particularly looking at PDEs as they arise in geophysical problems. Of course it would take some work to build appropriate PDE solvers from scratch. To save this effort we use the python PDE solver module `esys.escript`. This section will give an introduction into the work with `esys.escript` to solve 2D problems. To illustrate the use we will look at the 
calculation of gravity anomaly fields from subsurface density anomalies. The following section will then present some geophysical applications of PDEs. The the third section will discuss the use of `esys.escript` to solve seismic wave equations in time and in frequency domain.

We first present some details the PDEs for modeling gravity field anomalies before we start to
discuss how to use `esys.escript` to solve this PDE. 

## Useful links:

- [esys.escript home page](https://launchpad.net/escript-finley)
- [Researchgate](https://www.researchgate.net/project/esys-escript)
- [user's guide](https://launchpad.net/escript-finley/3.0+/5.4/+download/docs.zip)
- [API documentation](https://esys-escript.readthedocs.io/en/latest/index.html)


## An example problem: Gravity Field Anomalies
The gravitational field $\mathbf{g}$ (also called gravitational acceleration) is a vector field defining the gravitational force experienced by a particle as its mass multiplied by the gravitational field at that point.
In Cartesian coordinates the gravitational field $\mathbf{g}$ is expressed in the form
\begin{equation} 
\mathbf{g}=(g_x, g_y, g_z)
\end{equation} 
The vector $\mathbf{g}$ needs to fulfill the Gauss's law which is a generalization of Newton's law introduced in The Gauss's law is stated in the following way:
\begin{equation}\label{EQGAUSSLAW}
 \frac{\partial g_x}{\partial x} +  \frac{\partial g_y}{\partial y} + \frac{\partial g_z}{\partial z} = - 4\pi G\rho 
\end{equation}
where $G=6.67 \cdot 10^{-11}  \frac{m^3}{kg s^2}$ is the gravitational constant and $\rho$ is the density distribution.  Gauss's law also stated using the divergence operator $\mathbf{\nabla}^t$:
\begin{equation}
\mathbf{\nabla}^t \; \mathbf {g} =-4\pi G\rho 
\end{equation}
The magnetic field $\mathbf{g}$ is obtained from its potential $u$ 
using the *Grad* operator $\mathbf{\nabla}$:
\begin{equation} 
\mathbf{g} = -
\mathbf{\nabla} U 
\end{equation} 
with the gravity accelerations 
\begin{equation}\label{EQGRADRULE}
g_x =  - \frac{\partial U}{\partial x}, g_y =  - \frac{\partial U}{ \partial y} \mbox{ and } g_z =  - \frac{\partial U}{\partial z}, 
\end{equation} 
If $\rho$ in \eqref{EQGAUSSLAW} is selected as a density anomaly, ie. as the deviation from a constant background
density, then the Gauss's law \eqref{EQGAUSSLAW} and scalar potential definition \eqref{EQGRADRULE}
defines a PDE for the potential $u$ which gradient gives the gravity field anomaly $\mathbf{g}$
due to the density anomaly $\rho$. To model field observations we are interested in the 
vertical component $g_z$ of $\mathbf{g}$.


## The PDE template
In `esys.escript` the PDE to be solved is defined trough a generic PDE template 
in $x_0x_1$-coordinates. 
which is provided through the `LinearSinglePDE` class. The
template fits nicely with the problem of finding the gravity potential. We will in the $x_0=x$
and $x_1=z$ coordinate system. 

First we write `LinearSinglePDE` template down in an abstract formulation: When $u$ is the unknown
we define the so-called `flux` vector $\mathbf{F}$ which is in essence the negative gradient of the solution 
times some matrix $\mathbf{A}$ plus some vector $X$:
\begin{equation} \label{EQFLUX}
\mathbf{F} = - \mathbf{A} \mathbf{\nabla} u +\mathbf{X}
\end{equation} 
Ignoring the matrix $\mathbf{A}$ and setting $\mathbf{X}=0$ 
this already looks like the scalar potential definition \eqref{EQGRADRULE} when $u=U$ is the gravity potential and
$\mathbf{F}=\mathbf{g}$ is the gravity acceleration. 

The flux vector $\mathbf{F}$ needs to fulfill the conservation equation : 
\begin{equation}\label{EQCONSERVATION}
\mathbf{\nabla}^t \; \mathbf{F} + D \; u = Y 
\end{equation}
where $D$ is a scalar and $Y$ is the right hand side. We can easily identify Gauss's law \eqref{EQGAUSSLAW}
when we choose $D=0$ and $Y=- 4\pi G\rho$. 

Before we set this up in python we look at these equations in a bit more detail. For the 2D case the flux definition \eqref{EQFLUX} reads as
\begin{equation}\label{EQFLUX2}
\mathbf{F} = 
\begin{bmatrix}
 F_0 \\
 F_1
\end{bmatrix}
= - 
\begin{bmatrix}
A_{00} \frac{\partial u}{\partial  x_0} & + & A_{01} \frac{\partial  u}{\partial  x_1}\\
A_{10} \frac{\partial  u}{\partial  x_0} & + & A_{11} \frac{\partial  u}{\partial  x_1}
\end{bmatrix}
\end{equation} 
Comparison with the grad rule \eqref{EQGRADRULE} shows that we need to choose 
\begin{equation}
\begin{bmatrix}
A_{00} & A_{01} \\
A_{10}  & A_{11} 
\end{bmatrix}= 
\begin{bmatrix}
1 & 0  \\
0  & 1 
\end{bmatrix}
\end{equation}
To define the PDE we want to solve we need to define the coefficients in the PDE template
through `LinearSinglePDE` instance. To get the solution $u$ (in fact a numerical approximation is calculated)
is then as easy as calling the `getSolution` method. 
Before we can define the PDE we need to set up the region over which we would like to solve the PDE. In the
terminology of `esys.escript`  this is called a `Domain`. 

## How to create a domain

The first step to set-up the problem is to define the region over which the problem is solved. Here we use 
a rectangular grid of `NEx` elements in the horizonal x-direction and `NEz` elements in the vertical z-direction. 
We use a grid line spacing of `dx`. 

In [1]:
#%matplotlib notebook

In [2]:
NEx=100 # number of cells
NEz=100
dx=10. # in meter [m] grid spacing
Lx=dx*NEx
Lz=dx*NEz
print("domin extension = %s x %s"%(Lx,Lz))
print("grid = %s x %s"%(NEx,NEz))

domin extension = 1000.0 x 1000.0
grid = 100 x 100


We use the package `finley` which is part of the `esys.escript` distribution to set up the domain:

In [3]:
from esys.escript import *
from esys.finley import Rectangle
domain=Rectangle(l0=Lx, l1=Lz, n0=NEx, n1=NEz)
print(type(domain))


<class 'esys.finley.finleycpp.FinleyDomain'>


**Note** the following rules for `Rectangle`:
- the axis are labeled `x0` and `x1`
- the lower, left corner has the coordinates `(0.,0.)`.

Use `Brick` from `esys.finley` for 3D domains.
There are other domain packages available:
- `esys.finley` - general FEM solver solver also support unstructured meshes and contact elements
- `esys.ripley` - special solver for rectangular grids
- `esys.specley` - spectral element solver for wave problems (will discussed later)
- `esys.dudley` - FEM solver for tetrahedral and triangular meshes



## Setting up and solving a PDE

The first step to set up a PDE is to create an instance of the `LinearSinglePDE` class.
Here we call this instance `model` and attach it to the domain `domain` we have already created:

In [4]:
from esys.escript.linearPDEs import LinearSinglePDE

model=LinearSinglePDE(domain)
print(model)

<LinearPDE 140375848399536>


Now we need to set the coefficients $\mathbf{A}$ and $Y$. `esys-escript` will automatically assume that the other coefficients $\mathbf{X}$ and $D$ are zero. Let's start with coefficient $\mathbf{A}$:

Recall that for the Gauss' Law $\mathbf{A}$ is the identity matrix. `esys-escript` 
provides the convenience function `identityTensor` that sets up the identity matrix.
Its return value can directly be passed on to `model` as coefficient `A`:


In [5]:
model.setValue(A=identityTensor(domain))

**Note:**
`identityTensor` returns a $d \times d$ matrix where $d$ is chosen as the spatial 
of its argument. As `domain` is two dimensional a $2 \times 2$ identity matrix is returned.
If the domain becomes 3D at a later point the identity matrix will be defines as $3 \times 3$. This allows writing code that is identity from the spatial dimension of the domain.

Next we define the PDE coefficient $Y$ which is given as $Y=- 4\pi G\rho$ and requires us 
to define the density $\rho$. 
As an example we assume that the density anomaly is a circle centered
at $\mathbf{c}=(c_0, c_1)=(\frac{Lx}{2}, \frac{Lx}{3})$ with radius $R_C=100$m. 

To define $\rho$ we first need to calculate the distance of any point $\mathbf{x}$ in the 
domain from the $\mathbf{c}$. Then $rho$ is set to $\rho_0=1000 \frac{kg}{m^3}$ 
at those points which distance is smaller than $R_C$. For the others  $\rho$ is set to zero.

First gets the coordinates of the points in the domain: 

In [6]:
x=domain.getX()
print(type(x))

<class 'esys.escriptcore.escriptcpp.Data'>


`x` is a `Data` object which is giving locations in the domain. So `x` has two components: 

In [7]:
print("shape of data obect `x`",x.getShape())

shape of data obect `x` (2,)


Components of `x` can be accessed by slicing:

In [8]:
print("x0 coordinates = ",x[0])
print("x1 coordinates = ",x[1])

x0 coordinates =  Summary: inf=0 sup=1000 data points=10201
x1 coordinates =  Summary: inf=0 sup=1000 data points=10201


The statement `inf=0` gives the smallest value - in our case of the $x_0$ coordinate. 
The maximal value is shown as `sup=1000` which gives the value of `Lx`. The number of
point being used is `10201`. The reason for this is that there is one value for each grid point which would be '(n0+1) x (n1+1)' values (Why?). In our case this would be 
'(n0+1) x (n1+1) = (NEx+1) x (NEz+1) = 201 x 201 = 10201' values.


The distance of point $\mathbf{x}$ to point $\mathbf{c}$ can be calculated by 
\begin{equation}\label{eq:distance}
d=\sqrt{ (x_0-c_0)^2 + (x_1-c_1)^2}
\end{equation}

In [9]:
c=[Lx/2., Lz/3.]
RC=100. 

In [10]:
d=sqrt((x[0]-c[0])**2+(x[1]-c[1])**2)
print("distance to c = %s :"%c,d)

distance to c = [500.0, 333.3333333333333] : Summary: inf=3.33333 sup=833.333 data points=10201


Notice that `d` is also a `Data` object as it has been  derived from 
the `Data` object `x`. It can be seen as a function of the location `x`. 

There is a more compact form to calculate `d` using the `length` function:

In [11]:
d=length(x-c)
print("distance to c = %s :"%c,d)

distance to c = [500.0, 333.3333333333333] : Summary: inf=3.33333 sup=833.333 data points=10201


We want have the anomaly distribution `rho` to be `rho0` where the distance `d` is smaller than `RC` 
(or `d-RC<0`) and zero otherwise. We use the `whereNegative` function which returns one where its argument is negative and zero elsewhere:  

In [12]:
rho0=1000.
rho=rho0*whereNegative(d-RC)

Now we can set the right hand side `Y` of the PDE `model`:

In [13]:
import numpy as np
G=6.67e-11  # m^3/kg/sec^2 gravity constant
model.setValue(Y=-4*G*np.pi*rho)

We are expecting the gravity to vanish at large distances away from the circle. For a bounded domain as we use in the gravity model here we enforce this condition on the boundary of the domain formed by the bottom and top face
$x_1=0$ and $x_1=L_z$. To tell this to `LinearSinglePDE` we need to set a mask `q` which marks the locations on the surface where we want the solution $u$ to be zero:
\begin{equation}\label{EQQ}
q(\mathbf{x}) = \begin{cases}
>0  & \mbox{ set } u(\mathbf{x})=0 \\
=0  & \mbox{ no constraint at } x
\end{cases} 
\end{equation}
We use the `whereZero` function which takes a `Data` object. It returns 
a `Data` object which has the value 
**one** at location where the argument has the value **zero**. Otherwise zero is used. 
It is applied to define a `q` for each face of the domain. These `q`s are then combined
by addition to define the final `q` which is has positive value at any point on the 
surface of the domain but zero in the interior.

In [14]:
x=domain.getX()
q_bottom=whereZero(x[1])     # 1 for face x_1=0
q_top=whereZero(x[1]-Lz)  # 1 for face x_1=Lz

model.setValue(q=q_bottom+q_top)

Question is: What boundary conditions are applied on the other two faces?
By default the so called *weak* boundary conditions
\begin{equation}\label{eq:weakBC}
\mathbf{F} \cdot \mathbf{n} = F_0 n_0 + F_1 n_1 = 0
\end{equation}
with outer normal field $\mathbf{n}=(n_0, n_1)$ are assumed.
As $\mathbf{n}=(-1,0)$ on the left face and 
As $\mathbf{n}=(1,0)$ on the right face 
of the domain the boundary condition there become 
\begin{equation}\label{eq:weakBC1}
F_0= -\frac{\partial u}{\partial x_0} = 0
\end{equation}


Now we are are ready to get the solution of the PDE:

In [15]:
u=model.getSolution()

**Note**

The `getSolution` call involves the solution of system of linear equation. The dimension 
is the number of grid points. For large grids and for 3D problems this solve can take some time.

The `grad` function returns the gradient of the argument. The negative of the gradient 
gives us the gravitational acceleration `g`:

In [16]:
g=-grad(u)
print(g)

Summary: inf=-3.85762e-05 sup=4.6591e-05 data points=40000


The gravitational acceleration is a vector:

In [17]:
g.getShape()

(2,)

Before we start looking into postprocessing techniques we need to understand better 
what `esys.escript.Data` objects such as `d`, `u` and `g` are actually represent. 

## What do the values in `Data` objects actually mean?

When we print `Data` objects this shows the minimum (`inf`) and maximum (`sup`) value but also
the number of `data points` being used. Let have a look at the `d`, `u` and `g` again:

In [18]:
print("x :", x)
print("d :", d)
print("u :", u)
print("g :", g)

x : Summary: inf=0 sup=1000 data points=10201
d : Summary: inf=3.33333 sup=833.333 data points=10201
u : Summary: inf=-0.00974452 sup=0 data points=10201
g : Summary: inf=-3.85762e-05 sup=4.6591e-05 data points=40000


The number of data points for `x`, `d` and `u` are the same namely `10201`. As already explained this 
comes from the fact that `d` holds the data on the grid nodes (intersects of grid lines). 
The gradient `g` is stored cell (or element) based where in four integration points in each cell is used. 
This explains the number of data points of `40000` as 
\begin{equation}
NEx \times NEz \times \mbox{ # integration points } = 100  \times 100  \times 4 = 40000
\end{equation}
When handling `Data` objects it is obviously important to know which representation locations 
a particular `Data` object is 
using. For instance a `Data` object using grid nodes and a `Data` objects using cells cannot easily be added together. In order to handle this the `Data` object has an attribute that defines the way its values are handled.
This location attribute can be obtained by the `getFunctionSpace` method:

In [19]:
print("d is stored at ", d.getFunctionSpace())
print("g is stored at ", g.getFunctionSpace())

d is stored at  Finley_Nodes [ContinuousFunction(domain)] on FinleyMesh
g is stored at  Finley_Elements [Function(domain)] on FinleyMesh


This a list of  location attributes are available:

- `ContinuousFunction(domain)`: mesh nodes
- `Solution(domain)`: solution of a PDE, typically also on mesh nodes 
- `Function(domain)` : on integration points per element
- `ReducedFunction(domain)`: on element centers
- `DiracDeltaFunction(domain)`: point sources and sinks which we will use later

The locations where the values are hold can be obtained by the `getX` methof of a `FunctionSpace`:

In [20]:
X=g.getFunctionSpace().getX()

One can test if the `FunctionSpace` attribute of two `Data` objects is the same:

In [21]:
print("g and d on the same location?", g.getFunctionSpace() == d.getFunctionSpace())
print("X and g on the same location?",X.getFunctionSpace() == g.getFunctionSpace())
print("g is on integration points?",X.getFunctionSpace() == Function(domain))

g and d on the same location? False
X and g on the same location? True
g is on integration points? True


Interpolation can be used to change the data location. We would like to have the 
vertical gravity $g_z$ at element centers. Here we interpolate the `g[1]` which is the
vertical gravity at the integration points to the element centers: 

In [22]:
gz=interpolate(g[1], ReducedFunction(domain))
print(g)
print(gz)

Summary: inf=-3.85762e-05 sup=4.6591e-05 data points=40000
Summary: inf=-3.69606e-05 sup=4.65409e-05 data points=10000


**WARNING:** Not all interpolations work. This depends on the PDE model being used.
For instance interpolation from element centers to nodes is not supported for a `finley` domain:

In [23]:
interpolate(gz, ContinuousFunction(domain))

ValueError: No interpolation with data on elements with reduced integration order possible.

## Visualization using `matplotlib`

`matplotlib` is a powerful and very versatile tool for plotting data in python. 
Here we want to use it plot the distribution of the vertical gravity over the domain.
To hand `Data` objects to `matplotlib` they first need to converted into `numpy` arrays. 
With `convertToNumpy` `esys.escript` provides an easy mechanism to do this. 
To plot data we need not only the data but also the location at which they are position in order
to plot their distribution. So we not only convert the `Data` objects but also the corresponding 
locations which we can get through the `FunctionSpace` attribute:

In [ ]:
gz_np=convertToNumpy(gz)
x_np=convertToNumpy(gz.getFunctionSpace().getX())
print(gz_np)
print(x_np)

We now can use the `tricontourf` to plot filled contours and `tricontour` to plot contour lines of the
the spatial distribution of vertical gravity `gz` using its `numpy` version `gz_np` and the corresponding data locations `x_np`. 

** Note ** For more options see the documentaions of [tricontour](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.tricontour.html) and [tricontourf](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.tricontourf.html).


In [ ]:
tomGal = 100000. # conversion from m/s**2 to mGal

In [33]:
import matplotlib.pyplot as plt
tomGal = 100000

plt.figure(figsize=(7,7))
plt.clf()
contour=plt.tricontourf(x_np[0], x_np[1], gz_np[0]*tomGal, 15)
plt.tricontour(x_np[0], x_np[1], gz_np[0]*tomGal, 15, linewidths=0.8, colors='k')

plt.xlabel('$x_0$ [m]')
plt.ylabel('$x_1$ [m]')
plt.title("Vertical gravity $g_z$ [mGal] due to a circular anomaly")
plt.colorbar(contour)
plt.gca().set_aspect('equal')

NameError: name 'x_np' is not defined

We can also do this in 3D:

In [ ]:
if True:
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    plt.clf()
    ax = fig.gca(projection='3d')
    contour=ax.plot_trisurf(x_np[0], x_np[1], gz_np[0]*tomGal , linewidth=0.2, antialiased=True, cmap=plt.cm.CMRmap)
    plt.xlabel('$x_0$ [m]')
    plt.ylabel('$x_1$ [m]')
    plt.title("Vertical gravity $g_z$ [mGal]")
    plt.colorbar(contour)
    plt.show()

**Note**:
For larger meshes with millions of element and also for 3D domains `matplotlib` is not really suitbale. 
Firstly its is just too slow but it also does not provide appropriate functionalities to deal with these kind of data. Alternative packages are 
- [mayavi](https://docs.enthought.com/mayavi/mayavi/)
- [paraview](https://www.paraview.org/)
- [visit](https://wci.llnl.gov/simulation/computer-codes/visit/)

Although these packages can also be used from python it is more appropriate to use them separately trough their respective GUIs. To hand data to them `Data` objects are written to external files preferable in the `VTK` format. 
This can be done using the `saveVTK` function from the `esys.weipa` module, see the users guide for details.

## Value Picking

Visualization of the distribution is often not sufficient for instance if one wants to apply a more quantitative 
analysis of the result; for instance comparison with a observations. There is a mechanism to pick values of `Data` objects at specific locations. 

### Single Value
Here we want to obtain the vertical gravity at a point at a height $h=300$ m above the center of the circular anomaly located at $c$. The location of this point $\mathbf{p}$ is then
\begin{equation}\label{eq:P}
\mathbf{p}=(c_0, c_1+300)
\end{equation}
We define a so-called `Locator` that provides a mechanism to extract the value at this point $\mathbf{p}$.
The `Locator` is build for the use at all `Data` objects with a specific `FunctionSpace` attribute; in our case
that should be the same as for `gz`:

In [24]:
from esys.escript.pdetools import Locator
p=[c[0], c[1]+300]
point_locator=Locator(where=ReducedFunction(domain), x=p)

Now we can easily get the value of `gz` at `p`:

In [25]:
v=point_locator.getValue(gz)
print("value of gz @ %s is %s mGal. "%(point_locator.getX(), v*tomGal))

NameError: name 'tomGal' is not defined

The `Locator` actually is not always using the specified point but picks the location 
of the `FunctionSpace` nearest to the requested. In our case the point is actually moved by about $5$ m:

In [26]:
print("target point was %s."%(p,))

target point was [500.0, 633.3333333333333].


The `point_locator` can be reused to pick values from other `Data` objects as long as they can interpolated
to `ReducedFunction`:

In [27]:
v2=point_locator.getValue(g)
print("value of g @ %s is %s."%(point_locator.getX(), v2))

value of g @ [ 495.  635.] is [  1.67666900e-07  -1.32564753e-05].


### Along a line of points (transect)
One can also use the `Locator` to pick data for a set of points for instance along a horizontal transect.
First we define the $x_0$-coordinates of the points we would like to use:

In [28]:
x0_transect=np.linspace(0., Lx, NEx)

We then add the $x_1$ coordinate as $c_1+300$ to define the locations in the transect in the 2D domain:

In [29]:
h=300
x_transect=[ (x0, c[1]+h) for x0 in x0_transect]
print(c[1]+h-Lz/2)

133.33333333333326


Now we can create new `Locator` named `transect_locator` using the points `x_transect`:

In [30]:
transect_locator=Locator(where=ReducedFunction(domain), x=x_transect )

Then the vertical gravity across the transect can be picked from `gz`. We also get the true $x_0$ coordinates 
of the points in the transect:

In [31]:
gz_transect=transect_locator.getValue(gz*tomGal)
x0_transect=[ x[0] for x in transect_locator.getX()]

NameError: name 'tomGal' is not defined

And finally we can plot the vertical gravity over the transect:

In [32]:
plt.figure(figsize=(5,5))
plt.clf()
plt.plot(x0_transect, gz_transect)
plt.xlabel('offset [m]')
plt.ylabel('$g_z$ [mGal]')
plt.title("gravity anomaly over transect @ height %g"%(c[1]+h))
plt.show()

NameError: name 'plt' is not defined